To get started, answer the following questions using just the fullsample.csv jobs dataset:

Calculate some descriptive statistics for how many jobs per hour are being completed. What does the completions per hour look like over the time span of the dataset? Are there weekly trends, and has it been increasing over the last year?

Does the job state affect completions per hour? i.e. if I only look at jobs with exit code 0:0 in the "COMPLETED" state, is that a similar number of completions per hour as with all jobs, failed or cancelled? This will indicate if the load on the scheduler is by user design or is a result of users not sufficiently testing their jobs before submitting very large arrays. We also expect that most job completions will be in the "production" partition, but is this actually true?

In [1]:
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta
import time
import re
import numpy as np

In [13]:
#Job list with only completed jobs
fullsample_completed = pd.read_csv('fullsample_completed')

In [14]:
fullsample_completed['BEGIN'].min()  #Earliest date is Oct. 1,2020 @ 12:03:08 AM
fullsample_completed['END'].max()    #Latest date is Oct. 7, 2021 @ 8:41:11 PM

'2021-10-07T20:41:11'

In [15]:
#changing to datetime
fullsample_completed['BEGIN'] = pd.to_datetime(fullsample_completed['BEGIN'])
fullsample_completed['END'] = pd.to_datetime(fullsample_completed['END'])

In [16]:
# hours elapsed from earliest date to latest date is 8924.63 rounded to nearest hundredth 
t1 = fullsample_completed['BEGIN'].min()
t2 = fullsample_completed['END'].max()
duration_in_hr = ((t2 - t1).total_seconds())/3600
round(duration_in_hr,2)

8924.63

In [17]:
# 7,375,084 completed jobs including all jobs from some array
8924.63/7375084

0.00121010553913691

In [18]:
# changing REQTIME and USEDTIME from string to time type
fullsample_completed['REQTIME'] = fullsample_completed['REQTIME'].str.replace("-", " days ")
fullsample_completed['USEDTIME'] = fullsample_completed['USEDTIME'].str.replace("-", " days ")
fullsample_completed['REQTIME'] = pd.to_timedelta(fullsample_completed['REQTIME'])
fullsample_completed['USEDTIME'] = pd.to_timedelta(fullsample_completed['USEDTIME'])

In [7]:
fullsample_completed.head()

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
0,30853133,COMPLETED,2021-08-06 11:36:09,2021-09-05 11:36:32,262144Mn,20604.62M,30 days 00:00:00,30 days 00:00:23,1,1,cgw-platypus,0:0
1,30858137,COMPLETED,2021-08-06 19:04:39,2021-09-05 19:04:53,204800Mn,57553.77M,30 days 00:00:00,30 days 00:00:14,1,32,cgw-tbi01,0:0
2,30935078,COMPLETED,2021-08-09 16:52:51,2021-09-07 20:52:55,65536Mn,20577.96M,29 days 04:00:00,29 days 04:00:04,1,8,cgw-platypus,0:0
3,31364111_2,COMPLETED,2021-08-17 07:45:07,2021-09-10 16:45:24,16384Mn,9733.43M,24 days 09:00:00,24 days 09:00:17,1,1,production,0:0
4,31364111_3,COMPLETED,2021-08-17 07:45:07,2021-09-06 16:17:34,16384Mn,9708.04M,24 days 09:00:00,20 days 08:32:27,1,1,production,0:0


Calculate some descriptive statistics for how many jobs per hour are being completed. What does the completions per hour look like over the time span of the dataset? Are there weekly trends, and has it been increasing over the last year?

In [10]:
pd.options.display.max_rows=1000
fullsample_completed.describe()

,END,REQTIME,USEDTIME,NODES,CPUS
count,7.375084e+06,7375084,7375084,7.375084e+06,7.375084e+06
mean,6.786617e+00,0 days 22:13:53.170580836,0 days 03:47:45.835547093,1.005266e+00,1.855386e+00
std,3.192982e+00,1 days 14:30:05.436064806,0 days 12:40:53.468232103,4.053500e-01,3.107857e+00
min,1.000000e+00,0 days 00:01:00,0 days 00:00:00,1.000000e+00,1.000000e+00
25%,4.000000e+00,0 days 01:30:00,0 days 00:02:38,1.000000e+00,1.000000e+00
50%,8.000000e+00,0 days 08:00:00,0 days 00:13:56,1.000000e+00,1.000000e+00
75%,9.000000e+00,1 days 00:00:00,0 days 01:28:17,1.000000e+00,1.000000e+00
max,1.200000e+01,31 days 16:00:00,34 days 23:15:05,5.000000e+02,1.280000e+02


In [27]:
fullsample_completed['JOBID'].nunique()
#date_as_index = fullsample_completed.set_index("END")
#date_as_index.groupby(date_as_index.resample("H")[])
#date_as_index.resample("H").mean()

7375084

In [36]:
date_as_index.groupby(date_as_index.index.year).count()


,JOBID,STATE,BEGIN,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
END,,,,,,,,,,,
2020,1599163,1599163,1599163,1599163,1599163,1599163,1599163,1599163,1599163,1599163,1599163
2021,5775921,5775921,5775921,5775921,5775921,5775921,5775921,5775921,5775921,5775921,5775921


In [37]:
date_as_index.groupby(date_as_index.index.month).count()


,JOBID,STATE,BEGIN,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
END,,,,,,,,,,,
1,465414,465414,465414,465414,465414,465414,465414,465414,465414,465414,465414
2,512262,512262,512262,512262,512262,512262,512262,512262,512262,512262,512262
3,623419,623419,623419,623419,623419,623419,623419,623419,623419,623419,623419
4,480129,480129,480129,480129,480129,480129,480129,480129,480129,480129,480129
5,408582,408582,408582,408582,408582,408582,408582,408582,408582,408582,408582
6,623273,623273,623273,623273,623273,623273,623273,623273,623273,623273,623273
7,557295,557295,557295,557295,557295,557295,557295,557295,557295,557295,557295
8,1182141,1182141,1182141,1182141,1182141,1182141,1182141,1182141,1182141,1182141,1182141
9,893210,893210,893210,893210,893210,893210,893210,893210,893210,893210,893210


In [38]:
date_as_index.groupby(date_as_index.index.day).count()


,JOBID,STATE,BEGIN,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
END,,,,,,,,,,,
1,311323,311323,311323,311323,311323,311323,311323,311323,311323,311323,311323
2,277362,277362,277362,277362,277362,277362,277362,277362,277362,277362,277362
3,341697,341697,341697,341697,341697,341697,341697,341697,341697,341697,341697
4,274168,274168,274168,274168,274168,274168,274168,274168,274168,274168,274168
5,243411,243411,243411,243411,243411,243411,243411,243411,243411,243411,243411
6,208102,208102,208102,208102,208102,208102,208102,208102,208102,208102,208102
7,228884,228884,228884,228884,228884,228884,228884,228884,228884,228884,228884
8,221624,221624,221624,221624,221624,221624,221624,221624,221624,221624,221624
9,249640,249640,249640,249640,249640,249640,249640,249640,249640,249640,249640


In [39]:
date_as_index.groupby(date_as_index.index.hour).count()


,JOBID,STATE,BEGIN,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
END,,,,,,,,,,,
0,308599,308599,308599,308599,308599,308599,308599,308599,308599,308599,308599
1,278121,278121,278121,278121,278121,278121,278121,278121,278121,278121,278121
2,238815,238815,238815,238815,238815,238815,238815,238815,238815,238815,238815
3,202955,202955,202955,202955,202955,202955,202955,202955,202955,202955,202955
4,198374,198374,198374,198374,198374,198374,198374,198374,198374,198374,198374
5,181509,181509,181509,181509,181509,181509,181509,181509,181509,181509,181509
6,169023,169023,169023,169023,169023,169023,169023,169023,169023,169023,169023
7,203498,203498,203498,203498,203498,203498,203498,203498,203498,203498,203498
8,251747,251747,251747,251747,251747,251747,251747,251747,251747,251747,251747


In [40]:
date_as_index.groupby(date_as_index.index.week).count()


<ipython-input-40-7b7d3437bdd2>:1: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  date_as_index.groupby(date_as_index.index.week).count()


,JOBID,STATE,BEGIN,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
END,,,,,,,,,,,
1,85813,85813,85813,85813,85813,85813,85813,85813,85813,85813,85813
2,83541,83541,83541,83541,83541,83541,83541,83541,83541,83541,83541
3,83030,83030,83030,83030,83030,83030,83030,83030,83030,83030,83030
4,171786,171786,171786,171786,171786,171786,171786,171786,171786,171786,171786
5,140279,140279,140279,140279,140279,140279,140279,140279,140279,140279,140279
6,60628,60628,60628,60628,60628,60628,60628,60628,60628,60628,60628
7,127027,127027,127027,127027,127027,127027,127027,127027,127027,127027,127027
8,184328,184328,184328,184328,184328,184328,184328,184328,184328,184328,184328
9,154942,154942,154942,154942,154942,154942,154942,154942,154942,154942,154942
